In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np

tqdm.pandas()

In [2]:
data = pd.read_csv("../ai_medreview/data/data.csv")

In [4]:
data.head(2)

,time,rating,free_text,do_better,surgery,pcn,sentiment_free_text,sentiment_score_free_text,sentiment_do_better,sentiment_score_do_better,free_text_len,do_better_len,rating_score,feedback_labels,improvement_labels,submission_id,respondent-id,emotion_free_text,emotion_do_better
0,2024-01-01 10:51:00,Very good,NaN,NaN,Scarsdale-Medical-Centre,Brompton-Health-PCN,neutral,0.0,neutral,0.0,4.0,0,5.0,NaN,NaN,mannual,upload,NaN,NaN
1,2024-01-01 15:20:00,Very good,NaN,NaN,Scarsdale-Medical-Centre,Brompton-Health-PCN,neutral,0.0,neutral,0.0,5.0,0,5.0,NaN,NaN,mannual,upload,NaN,NaN


In [5]:
data.shape

(8106, 19)

In [6]:
from ai_medreview.data import *

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the ch

In [7]:
raw_data = load_google_sheet()

2024-09-05 16:39:00.185 | INFO     | ai_medreview.utils:wrapper:16 - 🖥️    Started: 'load_google_sheet'
2024-09-05 16:39:03.328 | INFO     | ai_medreview.utils:wrapper:22 - ✅ Completed: 'load_google_sheet' ⚡️3.141898 sec


In [8]:
raw_data.head()

,submission_id,respondent-id,time,rating,free_text,do_better,pcn,surgery
0,mannual,upload,2024-01-01 10:51:00,Very good,Everyone kind and helpful,,Brompton-Health-PCN,Scarsdale-Medical-Centre
1,mannual,upload,2024-01-01 15:20:00,Very good,Nothing. It was all great,,Brompton-Health-PCN,Scarsdale-Medical-Centre
2,,,2024-01-01 17:47:25,Very good,Knowledgable doctors,All Likely,Brompton-Health-PCN,Earls-Court-Surgery
3,mannual,upload,2024-01-02 07:58:00,Good,"Denise was amazing, kind and helpful. The rece...",,Brompton-Health-PCN,Scarsdale-Medical-Centre
4,mannual,upload,2024-01-02 10:44:00,Very good,Appointment,,Brompton-Health-PCN,Scarsdale-Medical-Centre


In [9]:
raw_data.shape

(8106, 8)

In [10]:
data.shape

(8106, 19)

In [11]:
def get_person_names_with_transformers(text):
    # Check if the text is empty or not a string
    if not text or not isinstance(text, str):
        return []  # Return an empty list if the input is invalid or empty

    # Initialize a list to store person names
    person_names = []

    try:
        # Run the NER pipeline on the valid input text
        entities = ner_pipeline(text)

        # Iterate over detected entities
        for entity in entities:
            # Check if the entity is classified as a person
            if entity["entity_group"] == "PER":
                # Add the detected name to the list of person names
                person_names.append(entity["word"])
    except ValueError as e:
        # Log the error for debugging
        print(f"Error processing text: {text}")
        raise e

    return person_names

In [ ]:
full_list = []

# Using tqdm to wrap around the iterable
for text in tqdm(raw_data["free_text"], desc="Processing texts"):
    person_list = get_person_names_with_transformers(text)
    personlist_len = len(person_list)
    if personlist_len != 0:
        full_list.append(person_list)
    else:
        full_list.append(np.nan)

In [12]:
import numpy as np
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor


# Shared tqdm instance
def process_texts(raw_data_column, pbar):
    full_list = []
    for text in raw_data_column:
        person_list = get_person_names_with_transformers(text)
        personlist_len = len(person_list)
        if personlist_len != 0:
            full_list.append(person_list)
        else:
            full_list.append(np.nan)
        pbar.update(1)
    return full_list


raw_data_1 = raw_data["free_text"]
raw_data_2 = raw_data["do_better"]

# Calculate total length for combined progress bar
total_length = len(raw_data_1) + len(raw_data_2)

with tqdm(total=total_length, desc="Processing texts") as pbar:
    with ThreadPoolExecutor(max_workers=2) as executor:
        future1 = executor.submit(process_texts, raw_data_1, pbar)
        future2 = executor.submit(process_texts, raw_data_2, pbar)

    full_list_1 = future1.result()
    full_list_2 = future2.result()

Processing texts: 100%|███████████████████████████████████████████████████████████████████████████████████| 16212/16212 [16:42<00:00, 16.17it/s]


In [13]:
len(full_list_1)

8106

In [14]:
len(full_list_2)

8106

In [15]:
new1 = {"free_text_PER": full_list_1}
new_df1 = pd.DataFrame(new1)

In [16]:
new2 = {"do_better_PER": full_list_2}
new_df2 = pd.DataFrame(new2)

In [17]:
new_df1.head()

,free_text_PER
0,NaN
1,NaN
2,NaN
3,[Denise]
4,NaN


In [18]:
new_df1.shape

(8106, 1)

In [23]:
merged = pd.concat([data, new_df1, new_df2], axis=1, join="inner")

In [25]:
merged.head(100)

,time,rating,free_text,do_better,surgery,pcn,sentiment_free_text,sentiment_score_free_text,sentiment_do_better,sentiment_score_do_better,...,do_better_len,rating_score,feedback_labels,improvement_labels,submission_id,respondent-id,emotion_free_text,emotion_do_better,free_text_PER,do_better_PER
0,2024-01-01 10:51:00,Very good,NaN,NaN,Scarsdale-Medical-Centre,Brompton-Health-PCN,neutral,0.000000,neutral,0.0,...,0,5.0,NaN,NaN,mannual,upload,NaN,NaN,NaN,NaN
1,2024-01-01 15:20:00,Very good,NaN,NaN,Scarsdale-Medical-Centre,Brompton-Health-PCN,neutral,0.000000,neutral,0.0,...,0,5.0,NaN,NaN,mannual,upload,NaN,NaN,NaN,NaN
2,2024-01-01 17:47:00,Very good,NaN,NaN,Earls-Court-Surgery,Brompton-Health-PCN,neutral,0.000000,neutral,0.0,...,2,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-01-02 07:58:00,Good,PERSON was amazing kind and helpful The recept...,NaN,Scarsdale-Medical-Centre,Brompton-Health-PCN,positive,0.937987,neutral,0.0,...,0,4.0,Reception Staff Interaction,NaN,mannual,upload,admiration,NaN,[Denise],NaN
4,2024-01-02 10:44:00,Very good,NaN,NaN,Scarsdale-Medical-Centre,Brompton-Health-PCN,neutral,0.000000,neutral,0.0,...,0,5.0,NaN,NaN,mannual,upload,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2024-01-12 16:29:00,Good,NaN,NaN,Scarsdale-Medical-Centre,Brompton-Health-PCN,neutral,0.000000,neutral,0.0,...,0,4.0,NaN,NaN,mannual,upload,NaN,NaN,NaN,NaN
96,2024-01-12 17:46:00,Good,NaN,NaN,Scarsdale-Medical-Centre,Brompton-Health-PCN,neutral,0.000000,neutral,0.0,...,0,4.0,NaN,NaN,mannual,upload,NaN,NaN,NaN,NaN
97,2024-01-12 18:28:00,Neither good nor poor,I expect that I have been given some prescript...,NaN,Scarsdale-Medical-Centre,Brompton-Health-PCN,negative,0.815552,neutral,0.0,...,0,3.0,Prescriptions and Medication Management,NaN,mannual,upload,neutral,NaN,NaN,NaN
98,2024-01-13 11:53:00,Good,NaN,NaN,Scarsdale-Medical-Centre,Brompton-Health-PCN,neutral,0.000000,neutral,0.0,...,0,4.0,NaN,NaN,mannual,upload,NaN,NaN,NaN,NaN


In [26]:
merged.to_csv("../ai_medreview/data/data.csv", index=False)

In [ ]:
count_list = []

for index, row in raw_data.iterrows():
    free_text = row["free_text"]
    do_better = row["do_better"]
    full_text = f"{free_text} {do_better}"
    count_list.append(full_text)

In [ ]:
count_list

In [ ]:
len(count_list)

In [ ]:
full_list = []

for text in tqdm(count_list, desc="Processing texts"):
    person_list = get_person_names_with_transformers(text)
    personlist_len = len(person_list)
    if personlist_len != 0:
        full_list.append(person_list)
    else:
        full_list.append(np.nan)

In [ ]:
full_list

In [ ]:
new = {"full_text": count_list, "names": full_list}

new_df = pd.DataFrame(new)

In [ ]:
new_df

In [ ]:
count = 1
for text in new_df["names"]:
    text = str(text)
    if text != "nan":
        print(f"{count} {text}")
        count = count + 1